<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ycfs6s2CZt2gAfIVpryebTZsy-7bQBwB?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)




# Chroma Cloud : From Basics to a Mini RAG App

- **Goal**: Learn how to add, retrieve, and query text with Chroma cloud
- **What you'll do**:
  - Install and set up dependencies
  - Create a client and collection
  - Add and query documents (including `get_or_create` and `upsert`)
  - Build a simple RAG pipeline using OpenAI models

Tip: Run each cell in sequence. This notebook is designed for Google Colab.

### Install All required libraries and setup you OPENAI API KEY and set up [chromaDB](https://www.trychroma.com/) cloud database and token id

In [ ]:
!pip install chromadb openai --quiet

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
chroma_token = userdata.get('CHROMA_TOKEN')

## Chroma Basics

We mirror the steps from the reference:
1. Create a client
2. Create a collection
3. Add documents with ids
4. Query similar documents

Notes:
- Chroma can embed for you with defaults.
- `ids` must be unique strings.


In [ ]:
import chromadb

# 1. Create a Chroma Client
client = chromadb.HttpClient(
  ssl=True,
  host='api.trychroma.com',
  tenant='6b53ed41-c379-47fa-b956-cc402f6fced8',
  database='cloud_quickstart',
  headers={
    'x-chroma-token': f'{chroma_token}'
  }
)

In [ ]:
# 2. Create a collection
collection = client.create_collection(name="my_collection")

# 3. Add documents with unique IDs
collection.add(
    ids=["id1", "id2"],
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges",
    ],
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 104MiB/s]


In [ ]:
# 4. Query the collection
results = collection.query(
    query_texts=["This is a query document"],
    n_results=2,
)
results

{'ids': [['id1', 'id2']],
 'distances': [[1.2771862, 1.2883999]],
 'embeddings': None,
 'metadatas': [[None, None]],
 'documents': [['This is a document about pineapple',
   'This is a document about oranges']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

### Using get_or_create and upsert

- **Why**: Avoid recreating the collection or duplicating docs on reruns.
- **get_or_create**: Returns existing collection or creates one if missing.
- **upsert**: Insert or update documents by ID.


In [ ]:
collection = client.get_or_create_collection(name="customer-support-messages")


In [ ]:
results = collection.query(
    query_texts=["I accidentally deleted several project boards yesterday and thought I could restore them"], # Chroma will embed this for you
    n_results=2 # how many results to return
)

print(results)

{'ids': [['e8789e2b-3dbe-4c9f-a4de-2396ce4d1bb3', 'cd1b60e5-702b-4e71-9496-db261ce624b6']], 'distances': [[0.4520844, 0.59681433]], 'embeddings': None, 'metadatas': [[{'timestamp': '2025-06-06T19:44:15.941Z', 'unix_timestamp': 1749239055, 'from_email': 'troy.pfeffer18@example.com'}, {'from_email': 'pearlie.langworth64@example.com', 'unix_timestamp': 1749240323, 'timestamp': '2025-06-06T20:05:23.791Z'}]], 'documents': [['I accidentally deleted several project boards yesterday and thought I could restore them, but the backup option only shows data from two weeks ago. Is there a way to access a more recent snapshot or undo the deletion without losing the work my team added this week?', 'Yesterday I accidentally overwrote my main project board while importing a template, and all the tasks and comments vanished. How can I roll back to a previous version or restore a backup if I never manually triggered one?']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}

## Mini RAG App

We will:
- Embed a small corpus into Chroma
- Retrieve top-k docs for a user query
- Generate an answer with OpenAI using retrieved context

Architecture:
- `embed(texts) -> vectors`
- `store(vectors, metadata) -> collection`
- `retrieve(query) -> top_k docs`
- `answer(query, context) -> response`


In [ ]:
import os
from typing import List

import chromadb
from chromadb.utils import embedding_functions

rag_client = chromadb.HttpClient(
  ssl=True,
  host='api.trychroma.com',
  tenant='6b53ed41-c379-47fa-b956-cc402f6fced8',
  database='cloud_quickstart',
  headers={
    'x-chroma-token': f'{chroma_token}'
  }
)

# Use OpenAI embeddings via Chroma's helper
openai_api_key = os.environ.get("OPENAI_API_KEY")
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=openai_api_key,
    model_name="text-embedding-3-small"
)

rag_collection = rag_client.get_or_create_collection(
    name="rag_collection",
    embedding_function=openai_ef,
)

print("RAG collection ready.")


RAG collection ready.


### Load a small corpus

- Mix short documents with simple metadata
- Use stable, unique string IDs
- Upsert so you can re-run safely


In [ ]:
docs = [
    "Hawaii is known for pineapples and beautiful beaches.",
    "Florida is famous for oranges and sunny weather.",
    "Pineapple can be grown in tropical climates.",
    "Oranges are rich in vitamin C and grow well in warm regions.",
]

metas = [
    {"source": "wiki", "topic": "hawaii"},
    {"source": "wiki", "topic": "florida"},
    {"source": "notes", "topic": "pineapple"},
    {"source": "notes", "topic": "oranges"},
]

ids = [f"doc-{i}" for i in range(len(docs))]

rag_collection.upsert(documents=docs, metadatas=metas, ids=ids)
print("Inserted/updated", len(docs), "docs into rag_collection.")


Inserted/updated 4 docs into rag_collection.


### Retrieve and answer

- Retrieve top-k docs for a query
- Call OpenAI to produce an answer grounded in the retrieved context
- Keep prompts concise and include citations


In [ ]:
from typing import Dict

from openai import OpenAI

client = OpenAI()


def retrieve(query: str, k: int = 3) -> Dict:
    return rag_collection.query(query_texts=[query], n_results=k)


def build_context(results: Dict) -> str:
    docs = results.get("documents", [[]])[0]
    metadatas = results.get("metadatas", [[]])[0]
    parts = []
    for i, (d, m) in enumerate(zip(docs, metadatas), start=1):
        parts.append(f"[Doc {i}] {d}\n(Source: {m})")
    return "\n\n".join(parts)


def answer_query(query: str, k: int = 3, model: str = "gpt-4o-mini") -> Dict[str, str]:
    results = retrieve(query, k=k)
    context = build_context(results)
    prompt = (
        "You are a helpful assistant. Use the provided context to answer the question.\n"
        "If the answer isn't in the context, say you don't know.\n\n"
        f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    )

    chat = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Answer strictly based on the given context."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.2,
    )

    answer = chat.choices[0].message.content
    return {"answer": answer, "context": context}


In [ ]:
# Try it
qa = answer_query("Where are oranges commonly grown?", k=2)
print("Answer:\n", qa["answer"])
print("\nContext used:\n", qa["context"])


Answer:
 Oranges are commonly grown in warm regions.

Context used:
 [Doc 1] Oranges are rich in vitamin C and grow well in warm regions.
(Source: {'topic': 'oranges', 'source': 'notes'})

[Doc 2] Florida is famous for oranges and sunny weather.
(Source: {'source': 'wiki', 'topic': 'florida'})


## Tips and Notes

- **Unique IDs**: Keep `ids` stable to avoid duplicates and enable updates via `upsert`.
- **Embeddings model**: `text-embedding-3-small` is cheap and strong; upgrade to `text-embedding-3-large` for best accuracy.
- **Quality**: Better chunking and metadata improve retrieval.
- **Safety**: Never hardcode API keys in notebooks you share.
- **Reproducibility**: Use `get_or_create_collection` and `upsert` in examples for reruns.

Next steps:
- Add chunking, re-ranking (e.g., rerankers), and streaming answers.
- Swap the chat model or prompt for your domain.
